In [1]:
'''Facing issue while building the pytrec_eval python module
Alternatively will be using the trec evalution manually using the .exe'''
#!pip install --upgrade setuptools

#!pip install pytrec_eval

'Facing issue while building the pytrec_eval python module\nAlternatively will be using the trec evalution manually using the .exe'

In [2]:
'''Steps:
1. Import required python packages
2. Parse / read xml files of document and query
3. Pre-process the document to remove stopwords,tokens
4. Check for term frequency
5. Indexing
6. Inverted indexing
7. From the above vector formatted document, use the Vector space model for query and ranking
8. repat the same with BM25  retrevial model
9. repeat the same with some Langauge Model
10. eval using trec
'''

'Steps:\n1. Import required python packages\n2. Parse / read xml files of document and query\n3. Pre-process the document to remove stopwords,tokens\n4. Check for term frequency\n5. Indexing\n6. Inverted indexing\n7. From the above vector formatted document, use the Vector space model for query and ranking\n8. repat the same with BM25  retrevial model\n9. repeat the same with some Langauge Model\n10. eval using trec\n'

In [3]:
# All imports and module installation can be found here.
import os
import string
import re
import xml.etree.ElementTree as ET
import subprocess

import numpy as np
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
parsed_xml = ET.parse('cranfield-trec-dataset-main/cranfield-trec-dataset-main/cran.all.1400.xml')

ParseError: junk after document element: line 24, column 0 (<string>)

In [5]:
'''Since there are parse errors, alternative solution is to read the file and add root elemet to form a valid xml'''

# Common function to read the files
def read_files_from_path(file):
    with open(file) as f: #
        return f.read()


In [6]:
cran_all_data = read_files_from_path('cranfield-trec-dataset-main/cranfield-trec-dataset-main/cran.all.1400.xml')
cran_document_xml = ET.fromstring('<r>' + cran_all_data + '</r>')

In [7]:
query_document_xml = ET.parse('cranfield-trec-dataset-main/cranfield-trec-dataset-main/cran.qry.xml')

In [8]:
# Using this def from previous NLP assignments and lectures

def non_liginustic_preprocess(corpus):
    corpus = str(corpus).lower() # converting to lower for normizaltion
    corpus = re.sub('\[.*?\]', '', corpus) # removing brakets
    corpus = re.sub('https?://\S+|www\.\S+', '', corpus) # removing url/http
    corpus = re.sub('[%s]' % re.escape(string.punctuation), '', corpus) # Removing the punctuation
    corpus = re.sub('\n', '', corpus)  # removeing next lines
    corpus = re.sub('\w*\d\w*', '', corpus) 
    corpus = re.sub('#', '', corpus) # remove hastags
    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)   
    stemming = PorterStemmer() 
    en_stopwords = stopwords.words("english")
    lemma_tokens = tokenizer.tokenize(corpus)
    
     # Removing stopwords  and apply stemming
    preprocessed_lemma = [stemming.stem(lemma) for lemma in lemma_tokens if lemma not in en_stopwords]
    return preprocessed_lemma

In [9]:
# https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.find

document_collection = {docs.find('docno').text: [non_liginustic_preprocess(docs.find('title').text), non_liginustic_preprocess(docs.find('text').text)]
        for docs in cran_document_xml.findall('doc')}

In [10]:
query_collection = {tags.find('num').text.strip(): [non_liginustic_preprocess(tags.find('title').text)]
           for tags in query_document_xml.getroot().findall('top')}

In [11]:
# fecthing the uniques keys from title and texts
unique_words_list_from_documents = list({word for key in document_collection.keys() for word in document_collection[key][0] + document_collection[key][1]})

In [12]:
trems_matrix_document = np.array([[document_collection[document_key][1].count(word) + document_collection[document_key][0].count(word) for document_key in document_collection.keys()] for word in unique_words_list_from_documents])

In [13]:
# vector space model
# convert the words to vector format which acts as weights and signifes the importance.
# Ease to cacluate using mathemtical formual and most occuring terms or words in vocalbalury.

def tf_idf(docs, term_matrix):
    num_documents = len(docs)
    copy_term_matrix = term_matrix.copy()
    
#     # Calculate TF for each term-document pair
#       # Iterate each column (documents)
#           # calculate total no.of words in document
#           # calculate term frequency and store in temporay term matrix

    copy_term_matrix = np.array([copy_term_matrix[:, doc_column] / np.sum(term_matrix[:, doc_column]) if np.sum(term_matrix[:, doc_column]) != 0 else copy_term_matrix[:, doc_column] for doc_column in range(term_matrix.shape[1])]).T
   
    # using np.log for numerical stability
    # using np.count_nonzero to get the non zero values to avoid sparsity.
    inverse_document_frequency_matrix = np.log(num_documents / (np.count_nonzero(term_matrix, axis=1) + 1)) 
    # calculate IDF for each term
    idf = inverse_document_frequency_matrix.reshape(len(inverse_document_frequency_matrix), 1)  # reshape

    return copy_term_matrix * idf

In [14]:
tf_idf_vsm = tf_idf(document_collection,trems_matrix_document )

In [15]:
# using best of 25
# Formula changes to calucate the BM25 and most of the core logic is same as tf-idf
def tf_idf_bm25(docs, term_matrix, k=1.5, b=0.75):
    avg = np.sum(np.sum(term_matrix, axis=1), axis=0) / term_matrix.shape[1]

    copy_term_matrix = term_matrix.copy() 
    # comphresions making it confusing so using normal for loops, update later - TODO
    for row in range(term_matrix.shape[0]):
        for doc_column in range(term_matrix.shape[1]): 
            documentTotalWords = len(docs[str(doc_column + 1)][0]) + len(docs[str(doc_column + 1)][1])  # Calculate the total words in the document
            
            if documentTotalWords != 0:
                copy_term_matrix[row, doc_column] = (term_matrix[row, doc_column] * (k + 1)) / (term_matrix[row, doc_column] + k * (1 - b + (b * documentTotalWords / avg)))
            else:
                copy_term_matrix[row, doc_column] = 0

    qt = np.count_nonzero(term_matrix, axis=1)
    idf = np.log((term_matrix.shape[1] - qt + 0.5) / (qt + 0.5))
    idf = idf.reshape(len(idf), 1)

    return copy_term_matrix * idf

In [16]:
bm25 = tf_idf_bm25(document_collection, trems_matrix_document)

In [ ]:
# from numpy.linalg import norm
# def cosine_similar(vec1, vec2):
#     mul = norm(vec1) * norm(vec2)
#     return 0 if mul == 0 else np.dot(vec1, vec2) / mul # check for an empty document

In [17]:
def query_ranking(query, documents, vocab, ir_model, rank=50):    
    query_vector = np.zeros(len(vocab))# Initialize query_vector with np.zeros with the length of vocab
    for word in query: # iterating for each word in query and updating the frequency of word accordingly
        if word in vocab:
            query_vector[vocab.index(word)] += 1
    query_vector = query_vector.reshape(1, -1)  # Reshape the vector

    similarities = cosine_similarity(query_vector, ir_model.T).flatten()  # Transpose tfIDF for compatibility
#     print(similarities)
    top_rank_doc = np.argsort(similarities)[::-1][:rank]
    
    
    query_rank = []
    for rank, docno in enumerate(top_rank_doc):
        score = {}
        score["docno"] = docno + 1
        score["rank"] = rank + 1
        score["similarity"] = similarities[docno]
        query_rank.append(score)
    
    return query_rank

In [18]:
def eval_trec(filepath,runid,ir_model):
    if os.path.exists(filepath):
        os.remove(filepath)

    with open(filepath, 'w') as output_file:
        for q in query_collection.keys():
            # pass one query and documents to rank with
            results = query_ranking(query_collection[q][0], document_collection, unique_words_list_from_documents, ir_model)
            for result in results:
                queryId = q
                docno = result["docno"]
                rank = result["rank"]
                similarity = result["similarity"]
                output_file.write(f"{q} 0 {docno} {rank} {similarity} {runid}\n")
        
    command = 'trec_eval -m map -m P.5 -m ndcg cranqrel.trec.txt ' + filepath

    print(subprocess.getoutput(command))

In [19]:
eval_trec('vector_space_model_retrival.txt' , 'VSM', tf_idf_vsm)

map                   	all	0.0077
P_5                   	all	0.0158
ndcg                  	all	0.0259


In [20]:
eval_trec('bm25.txt' , 'BM25', bm25)

map                   	all	0.0067
P_5                   	all	0.0118
ndcg                  	all	0.0231
